In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import pickle

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.4f}".format)
pd.set_option("display.max_info_rows", 200)


import surprise
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD, SVDpp, BaselineOnly, NMF, CoClustering
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("hls", 15)

# Dataset

In [2]:
# List of all products from Sephora as of June 24 with binary columns for categories

products = pd.read_csv("df_sephora.csv")

In [3]:
products.info(null_counts=True, max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9007 entries, 0 to 9006
Data columns (total 192 columns):
id                                   9007 non-null object
name                                 9007 non-null object
brand_id                             9007 non-null int64
brand_name                           9007 non-null object
short_desc                           8997 non-null object
long_desc                            8997 non-null object
item                                 9007 non-null int64
item_name                            7696 non-null object
list_price                           9007 non-null float64
variation                            9007 non-null object
variants                             3930 non-null object
price_low                            9007 non-null float64
price_high                           9007 non-null float64
is_limited_edition                   9007 non-null int64
rating                               8792 non-null float64
review_count         

In [5]:
df = pd.read_csv("user_item_rating_raw.csv")
df = df[["uid", "author_id", "product_id", "rating"]].copy()

In [6]:
author_ids = list(df.uid.unique())
product_ids = list(df.product_id.unique())

In [8]:
products_short = products[products.id.isin(product_ids)][[
                        "id", "rating", "review_count", "categories"
                    ]].copy().sort_values(["rating", "review_count"], ascending=False)

products_short.shape

(6839, 4)

In [11]:
skincare = products_short[(products_short.categories.str.contains("cat150006", na=False))].copy()
skincare

,id,rating,review_count,categories
7693,P443816,5.0000,14,"{'cat60121': 'Beauty Supplements', 'cat1860031': 'Wellness', 'cat150006': 'Skincare'}"
7550,P442110,5.0000,11,"{'cat1120031': 'Face Oils', 'cat1230034': 'Moisturizers', 'cat150006': 'Skincare'}"
6858,P457002,5.0000,10,"{'cat60121': 'Beauty Supplements', 'cat1860031': 'Wellness', 'cat150006': 'Skincare'}"
2935,P455421,5.0000,7,"{'cat60099': 'Face Wash & Cleansers', 'cat1230033': 'Cleansers', 'cat150006': 'Skincare'}"
4954,P449607,5.0000,7,"{'cat60121': 'Beauty Supplements', 'cat1860031': 'Wellness', 'cat150006': 'Skincare'}"
...,...,...,...,...
1892,P442567,2.7143,56,"{'cat920041': 'Face Masks', 'cat1440038': 'Masks', 'cat150006': 'Skincare'}"
1987,P450138,2.5000,18,"{'cat920041': 'Face Masks', 'cat1440038': 'Masks', 'cat150006': 'Skincare'}"
795,P456991,2.5000,16,"{'cat920033': 'Face Sunscreen', 'cat60113': 'Sun Care', 'cat150006': 'Skincare'}"
4614,P456409,2.4857,35,"{'cat60109': 'Lip Balms & Treatments', 'cat1520035': 'Lip Treatments', 'cat150006': 'Skincare'}"


In [12]:
fragrance = products_short[(products_short.categories.str.contains("cat160006", na=False))].copy()
fragrance

,id,rating,review_count,categories
143,P442947,5.0000,18,"{'cat60148': 'Perfume', 'cat1230039': 'Women', 'cat160006': 'Fragrance'}"
1336,P453668,5.0000,15,"{'cat60148': 'Perfume', 'cat1230039': 'Women', 'cat160006': 'Fragrance'}"
566,P450560,5.0000,13,"{'cat60146': 'Perfume Gift Sets', 'cat1230041': 'Value & Gift Sets', 'cat160006': 'Fragrance'}"
437,P430174,5.0000,11,"{'cat60150': 'Cologne', 'cat1230040': 'Men', 'cat160006': 'Fragrance'}"
519,P418386,5.0000,11,"{'cat60150': 'Cologne', 'cat1230040': 'Men', 'cat160006': 'Fragrance'}"
...,...,...,...,...
8888,P431444,3.1600,25,"{'cat60148': 'Perfume', 'cat1230039': 'Women', 'cat160006': 'Fragrance'}"
4546,P452679,3.0667,15,"{'cat60146': 'Perfume Gift Sets', 'cat1230041': 'Value & Gift Sets', 'cat160006': 'Fragrance'}"
7809,P448141,3.0177,396,"{'cat60148': 'Perfume', 'cat1230039': 'Women', 'cat160006': 'Fragrance'}"
4062,P448464,2.8235,17,"{'cat1110034': 'Rollerballs & Travel Size', 'cat1230039': 'Women', 'cat160006': 'Fragrance'}"


In [14]:
makeup = products_short[(products_short.categories.str.contains("cat140006", na=False))].copy()
makeup

,id,rating,review_count,categories
7516,P454769,5.0000,37,"{'cat60049': 'Lipstick', 'cat180010': 'Lip', 'cat140006': 'Makeup'}"
2923,P449105,5.0000,15,"{'cat60067': 'Face Brushes', 'cat1520033': 'Brushes & Applicators', 'cat140006': 'Makeup'}"
4983,P435444,5.0000,15,"{'cat180012': 'Nail', 'cat140006': 'Makeup'}"
4581,P457818,5.0000,13,"{'cat60059': 'Lip Sets', 'cat180010': 'Lip', 'cat140006': 'Makeup'}"
2290,P437365,5.0000,10,"{'cat60030': 'Eyeshadow', 'cat130054': 'Eye', 'cat140006': 'Makeup'}"
...,...,...,...,...
1829,P432078,2.4077,130,"{'cat60026': 'Mascara', 'cat130054': 'Eye', 'cat140006': 'Makeup'}"
2139,P448503,2.2288,118,"{'cat60028': 'Eyeliner', 'cat130054': 'Eye', 'cat140006': 'Makeup'}"
4204,P434503,2.2084,715,"{'cat60028': 'Eyeliner', 'cat130054': 'Eye', 'cat140006': 'Makeup'}"
3577,P41983035,nan,0,"{'cat60020': 'Highlighter', 'cat130058': 'Face', 'cat140006': 'Makeup'}"


In [22]:
def age_range(age):

    age_dict = {
        "13to17": range(1, 17 + 1),
        "18to24": range(18, 24 + 1),
        "25to34": range(25, 34 + 1),
        "35to44": range(35, 44 + 1),
        "45to54": range(45, 54 + 1),
        "over54": range(54, 100 + 1),
        "0": [0]
    }

    if age > 100 or age < 0:
        print("please check age input")
        return "error"

    for key, val in age_dict.items():
        if age in val:
            age_group = key
            if age < 13:
                print("products may not be aged appropriate")
            return age_group

In [17]:
def get_topn(uid, product_ids, n=50):
    
    algo=pickle.load(open("SVD_best.pkl", "rb"))
    
    items_to_pred = [(uid, iid, 0) for iid in product_ids]
    
    predictions = algo.test(items_to_pred)
    
    item_predictions = pd.DataFrame({
        "iid": product_ids,
        "est": [pred.est for pred in predictions]
    }).sort_values("est", ascending=False).reset_index(drop=True)
    
    
    return item_predictions.head(n)

In [23]:
rel = str(input("Relationship to recipient: ")).lower()
budget_max = float(input("Maximum budget in US$: "))
budget_low = float(input("Minimum in US$: "))
age = int(input("Age (plese provide estimate if unknown): "))
stone = str(input("Skintone (enter 0 if unknown):")).lower()
stype = str(input("Skintype (enter 0 if unknown):")).lower()
sconcerns = str(input("Main Skinconcern (enter 0 if unknown):")).lower()
ecolor = str(input("Eye color (enter 0 if unknown):")).lower()
hcolor = str(input("Hair color (enter 0 if unknown):")).lower()
hcondition = str(input("Hair condition (enter 0 if unknown):")).lower()
hconcerns = str(input("Hair concerns (enter 0 if unknown):")).lower()

Relationship to recipient: client
Maximum budget in US$: 150
Minimum in US$: 75
Age (plese provide estimate if unknown): 34
Skintone (enter 0 if unknown):medium
Skintype (enter 0 if unknown):0
Main Skinconcern (enter 0 if unknown):aging
Eye color (enter 0 if unknown):brown
Hair color (enter 0 if unknown):black
Hair condition (enter 0 if unknown):0
Hair concerns (enter 0 if unknown):0


In [ ]:
def get_uid(age=0,
            stone=0,
            stype=0,
            sconcerns=0,
            ecolor=0,
            hcolor=0,
            hconditon=0,
            hconcerns=0):

    ager = age_range(age)
    if ager == "error":
        return "please reenter details"

    skintone_options = [
        'porcelain', 'fair', 'light', 'medium', 'tan', 'olive', 'dark', 'deep',
        'ebony'
    ]

    skintype_options = ["normal", "oily", "dry", "combination"]

    sconcerns_options1 = [
        "acne", "aging", "blackheads", "uneven skintones", "sensitivity",
        "dark circles", "calluses", "pores", "redness", "dullness",
        "cellulite", "sun damage", "cuticles", "stretch marks", "puffiness"
    ]

    sconcerns_options2 = [
        "acne", "aging", "blackheads", "unevenSkinTones", "sensitivity",
        "darkCircles", "calluses", "pores", "redness", "dullness", "cellulite",
        "sunDamage", "cuticles", "stretchMarks", "puffiness"
    ]

    ecolor_options = ['green', 'blue', 'brown', 'hazel', 'gray']
    
    hcolor_options = ['brunette', 'black', 'red', 'blonde', 'gray', 'auburn']
    
    
    
    hcondition_options2 = [
        'chemicallyTreated', 'normal', 'wavy', 'fine', 'curly', 'coarse',
        'dry', 'oily', 'straight'
    ]
    
    
    #     hconcerns_options =

    if stone not in skintone_options:
        return "please reenter details"

    if stype not in skintype_options:
        return "please reenter details"

    if sconcerns not in sconcerns_options1:
        return "please reenter details"
    else:
        idx = sconcerns_options1.index(sconcerns)
        sconcerns = sconcerns_options2[idx]

In [78]:
age = 36 
ager = age_range(age)
stone = "medium" # 'light' 'medium' 'fair' 'ebony' 'olive' 'deep' 'dark' 'tan' 'porcelain'
stype = "0" # 'combination' 'oily' 'normal' 'dry'
sconcerns = "0" # 'acne' 'aging' 'blackheads' 'unevenSkinTones' 'sensitivity' 
# 'darkCircles' 'calluses' 'pores' 'redness' 'dullness' 'cellulite'
# 'sunDamage' 'cuticles' 'stretchMarks' 'puffiness'
ecolor = "brown" # 'green' 'blue' 'brown' 'hazel' 'gray'
hcolor = "0" # brunette' 'black' 'red' 'blonde' 'gray' 'auburn'
hcondition = "0" #'chemicallyTreated' 'normal' 'wavy' 'fine' 'curly' 'coarse' 'dry'
# 'oily' 'straight'
hconcerns = "0"# 'ColorProtection' 'Damage' 'CurlyEnhancing' 'Dandruff' 'Volumizing'
#  'AntiAging' 'Frizz' 'Hold' 'Thinning' 'Oiliness' 'StraighteningSmoothing'
#  'HeatProtection' 'Shine'

budget_low = 50
budget_max = 100

uid_query=ager+"_"+stone+"_"+stype+"_"+sconcerns+"_"+ecolor+"_"+hcolor+"_"+hcondition+"_"+hconcerns.title()


items_to_pred = [[uid_query, iid, 0] for iid in product_ids]
predictions = algo.test(items_to_pred)
predictions

[Prediction(uid='35to44_medium_0_0_brown_0_0_0', iid='P17876556', r_ui=0, est=4.7335473163304504, details={'was_impossible': False})]

In [79]:
items_to_pred = [[uid_query, iid, np.nan] for iid in product_ids]
predictions = algo.test(items_to_pred)
pred_ratings = np.array([pred.est for pred in predictions])

# map rating preds to product id
top_products = pd.DataFrame(data=pred_ratings,
                            columns=["pred_ratings"],
                            index=product_ids).sort_values(
                                "pred_ratings",
                                ascending=False).reset_index().head(500)

# get product details
top_products = pd.merge(top_products,
                        products,
                        how="left",
                        left_on="index",
                        right_on="id")

del items_to_pred, pred_ratings, predictions

top_products = top_products[(top_products.price_low >= budget_low)
                            & (top_products.price_high <= budget_max)]
top_products.sort_values(
    ["pred_ratings", "rating", "review_count"], ascending=False).head(15).drop(
        columns=["long_desc", "brand_long", "ingredients", "suggested_usage"])

,index,pred_ratings,id,name,brand_id,brand_name,short_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
0,P459127,4.9411,P459127,Platinum Lip Plump SPF 30,6318,Dr. Lara Devgan Scientific Beauty,A lip plumper that improve the smoothness and hydration of the lips in the form of a clear gloss.,2374049,0.10 oz/ 3 mL,50.0000,None,NaN,50.0000,50.0000,0,4.9109,505,2652,"{'cat60109': 'Lip Balms & Treatments', 'cat1520035': 'Lip Treatments', 'cat150006': 'Skincare'}",https://www.sephora.com/product/dr-lara-devgan-scientific-beauty-platinum-lip-plump-P459127?skuId=2374049,https://www.sephora.com/productimages/sku/s2374049-main-zoom.jpg,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,P444401,4.9341,P444401,Solstice Eyeshadow Palette,6276,Aether Beauty,"A palette of 12 golden hour shades in matte, shimmer, satin, metallic, and duo-chrome finishes to have your eyes glowing like a sunset.",2189975,NaN,58.0000,None,NaN,58.0000,58.0000,0,4.9091,143,8835,"{'cat60045': 'Eye Palettes', 'cat130054': 'Eye', 'cat140006': 'Makeup'}",https://www.sephora.com/product/summer-solstice-eyeshadow-palette-P444401?skuId=2189975,https://www.sephora.com/productimages/sku/s218997